In [9]:
import requests
import pandas as pd
import urllib.request
from urllib.request import quote
from bs4 import BeautifulSoup

# 라이브 커머스 , 라방 키워드로 네이버 기사 크롤링 

def get_news(query, page_num = 10):

  news_df = pd.DataFrame(columns = ("Title", "press", "Datetime", "Article", "Link"))
  index = 0

  url_query =  quote(query)  # utf_8 로 인코딩 

  # 네이버 넘어가기 한계로 두번에 나누어서 진행 
  # 라방 라이브커머스 라이브쇼핑 
  # 첫번째 
  #url = "https://search.naver.com/search.naver?where=news&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds=2021.01.01&de=2022.12.09&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20210101to20221209&is_sug_officeid=0&query=" + url_query
  # 두번째 
  url = "https://search.naver.com/search.naver?where=news&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds=2022.04.29&de=2022.12.09&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20220429to20221209&is_sug_officeid=0&query=" + url_query

  
  for _ in range(0, page_num):
    serch_url = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(serch_url, 'html.parser')
    links = soup.find_all('div', {'class' : 'info_group'})
    
    for link in links:
          press = link.find('a', {'class' :"info press"}).get_text()
          news_url = link.find('a' ,{'class' :'info'}).get('href') # 뉴스 기사 가지고 오기  

          # 네이버 기사아 아닐경우를 대비해서 두번째 url이 없으면 그만둔다. 
          try:
              news_url = link.select('a.info')[1].get('href')
              news_link = urllib.request.urlopen(news_url).read()
              news_html = BeautifulSoup(news_link,'html.parser')

              title = news_html.find('h2' ,{'id':"title_area"}).get_text()
              datetime = news_html.find('span' , {'class':"media_end_head_info_datestamp_time _ARTICLE_DATE_TIME"}).get_text()
              # article = news_html.find('div' ,{'id': "dic_area"}).get_text()
              # article = article.replace('\n','')
              # article = article.replace('\t','')

              news_df.loc[index] = [title, press, datetime,news_url]
              index += 1

              # print("#", end="") 

          except:
              continue

    try:
      next = soup.find('a',{'class' :'btn_next'}).get('href')
      url = "https://search.naver.com/search.naver" + next
    except:
      break  
                
  return news_df #이게 왜 안먹히지?
        

        
        

In [2]:
# url 1번째
query = input('검색 질의 :')
news_live_1 = get_news(query, 2000)

In [ ]:
# url 2번째
query = input('검색 질의 :')
news_live_2 = get_news(query, 2000)

검색 질의 :라방 라이브커머스 라이브쇼핑


In [ ]:
news_live_2['Datetime'] = news_live_2['Datetime'].str.split(' ',1).str[0]
news_live_2['Datetime']  = pd.to_datetime(news_live_2['Datetime'])
news_live_2.to_csv('article_2000_2.csv', index = False ,encoding="utf-8-sig" )
news_live_2